### Packages

In [18]:
%pip install openai python-dotenv


Note: you may need to restart the kernel to use updated packages.


### Environment Variables and Paths

In [19]:
import os
import re
import openai
import json
from dotenv import load_dotenv
from pathlib import Path

from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found. Please set it in the .env file.")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Define the base directory
BASE_DIR = Path.cwd()

# Modules directory
MODULES_DIR = BASE_DIR / "Modules"

# Define output directory for skill maps
SKILL_MAP_DIR = BASE_DIR / "skill_map"
SKILL_MAP_DIR.mkdir(exist_ok=True)

print("Setup complete.")

Setup complete.


### Read Markdown

In [20]:
def read_markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


### Generate Detailed Skill Map

In [78]:
def generate_skill_map(module_number, learning_goals, previous_skills=None):
    """
    Generates a detailed skill map for a module using OpenAI's API.

    :param module_number: The current module number (e.g., M1, M2).
    :param learning_goals: The basic learning goals extracted from learning_goals.md.
    :param previous_skills: Accumulated skills from previous modules.
    :return: Detailed skill map as a string.
    """
    prompt = f"""
You are an expert in educational content creation for programming courses. Based on the following learning goals for Module {module_number}, please provide a detailed skill map. The skill map should have a well-defined structure as outlined below:

1. **General Description**: Provide an overview of the entire module, summarizing the core concepts covered.

2. **Main Points**: For each key concept covered in the module, provide a short general description. 

3. **Subpoints**: Under each main point, list bullet points that expand on specific details, such as subtopics, examples, or techniques relevant to that concept.

Please structure the skill map as follows:
- **General Description**: A summary of the module.
- **Main Point**: Description of the concept.
    - **Subpoints**: Bullet points explaining subtopics or details related to the main point.
    
Ensure the output follows this structured format.

**Basic Learning Goals:**
{learning_goals}

**Previous Skills:**
{previous_skills if previous_skills else "None"}

**Detailed Skill Map:**
"""

    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": "You are a university level teacher in programming."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.3,
        )
        # Corrected access using dot notation
        detailed_skill_map = response.choices[0].message.content.strip()
        return detailed_skill_map
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return None

###  Iterate Through Modules and Generate Skill Maps

In [ ]:
from collections import OrderedDict

# Initialize an ordered dictionary to keep track of accumulated skills
accumulated_skills = OrderedDict()

# Iterate through each module in order (sorted by module number)
for module_dir in sorted(MODULES_DIR.iterdir(), key=lambda x: x.name):
    if module_dir.is_dir():
        module_number = module_dir.name  # e.g., M1, M2, etc.
        print(f"Processing {module_number}...")

        # Path to learning_goals.md
        learning_goals_path = module_dir / "learning_goals.md"
        if not learning_goals_path.exists():
            print(f"  Skipping {module_number}: learning_goals.md not found.")
            continue

        # Read basic learning goals
        basic_learning_goals = read_markdown(learning_goals_path)

        # Get accumulated skills as a string
        previous_skills = "\n".join(accumulated_skills.values()) if accumulated_skills else None

        # Generate detailed skill map
        detailed_skill_map = generate_skill_map(module_number, basic_learning_goals, previous_skills)
        if detailed_skill_map:
            print(f"  Generated skill map for {module_number}.")
            # Store in accumulated_skills
            accumulated_skills[module_number] = detailed_skill_map
        else:
            print(f"  Failed to generate skill map for {module_number}.")


Processing M1...


### Parse Skill Map

In [73]:
def parse_skill_map(detailed_skill_map):
    """
    Parses the detailed skill map text into a hierarchical dictionary.
    
    :param detailed_skill_map: The detailed skill map as a string.
    :return: Parsed skill map as a dictionary with general description and detailed points.
    """
    skill_map = {"general_description": "", "detailed_points": []}
    lines = detailed_skill_map.split('\n')

    current_main_point = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("**General Description**"):
            # General description section
            skill_map['general_description'] = stripped_line.replace("**General Description**:", "").strip()
        elif stripped_line.startswith("**Main Point**"):
            # New main point
            if current_main_point:
                skill_map['detailed_points'].append(current_main_point)
            current_main_point = {"main_point": stripped_line.replace("**Main Point**:", "").strip(), "sub_points": []}
        elif stripped_line.startswith('- ') and current_main_point:
            # Sub-bullet point for the current main point
            sub_point = stripped_line.lstrip('- ').strip()
            current_main_point['sub_points'].append(sub_point)

    # Add the last main point if exists
    if current_main_point:
        skill_map['detailed_points'].append(current_main_point)

    return skill_map


In [74]:
# Iterate through accumulated_skills and save each as a JSON file
for module_number, detailed_skill_map in accumulated_skills.items():
    # Parse the generated skill map to extract general description and detailed points
    parsed_skill_map = parse_skill_map(detailed_skill_map)
    
    # Define the JSON file path where the skill map will be saved
    json_file_path = SKILL_MAP_DIR / f"{module_number}.json"
    
    # Save the parsed skill map as a JSON file
    try:
        with open(json_file_path, 'w', encoding='utf-8') as json_file:
            # Dump the parsed skill map dictionary into the JSON file with indentation for readability
            json.dump(parsed_skill_map, json_file, indent=4)
        # Print success message
        print(f"Saved skill map for {module_number} to {json_file_path}.")
    except Exception as e:
        # Print error message if saving fails
        print(f"  Error saving {module_number} to JSON: {e}")


Saved skill map for M1 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M1.json.
Saved skill map for M2 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M2.json.
Saved skill map for M3 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M3.json.
Saved skill map for M4 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M4.json.
Saved skill map for M5 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M5.json.
Saved skill map for M6 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M6.json.
Saved skill map for M7 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M7.json.
Saved skill map for M8 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M8.json.
Saved skill map for M9 to /Users/avidfayyaz/KTH/TA/tests/inda-xx/SkillMap-Extract/skill_map/M9.json.
